## Rastrigin Function

In mathematical optimization, the Rastrigin function is a non-convex function used as a performance test problem for optimization algorithms. It is a typical example of non-linear multimodal function. It was first proposed in 1974 by Rastrigin as a 2-dimensional function and has been generalized by Rudolph. Finding the minimum of this function is a fairly difficult problem due to its large search space and its large number of local minima.[wiki]

In [1]:
import numpy as np
import matplotlib as pltm
import matplotlib.pyplot as plt
from matplotlib import cm 
from mpl_toolkits.mplot3d import Axes3D 

pltm.use('TkAgg')

X = np.linspace(-5.12, 5.12, 100)     
Y = np.linspace(-5.12, 5.12, 100)     
X, Y = np.meshgrid(X, Y) 

Z = (X**2 - 10 * np.cos(2 * np.pi * X)) + (Y**2 - 10 * np.cos(2 * np.pi * Y)) + 20

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.nipy_spectral, linewidth=0.08, antialiased=True)

plt.show()

#plt.savefig('rastrigin_graph.png')

### Homework Assignment

Use the Particle Swarm Optimization Algorithm to optimize the Rastrigin Function

- 2 dimension
- 3 dimension
- 4 dimension

Run the PSO on the Rastrigin Function. Compile the results in table with different iterations, particles, etc. Repeat the results with different implementations of v, c1 (a),c2 (b) as found in the lecture notes in Optimization_PSO slides. Refer to the lecture slides on example of tables required. Exactly what values to choose will be up to you. The objective is to investigate the effect of the variables and attributes of PSO on optimization of the Rastrigin function in multiple dimensions. 

Comment on the results.

Please submit a PDF/Word file presenting your results, comments and discussion.

# Nabil Bachroin (M11107814)

In [2]:
import numpy as np

In [3]:
def rastrigin(x):
    A = 10
    return A * len(x) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in x])

In [4]:
class Particle:
    def __init__(self, dim):
        self.position = np.random.uniform(-5.12, 5.12, dim)
        self.velocity = np.random.uniform(-1, 1, dim)
        self.best_position = np.copy(self.position)
        self.best_value = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        r1 = np.random.random(self.position.shape)
        r2 = np.random.random(self.position.shape)
        cognitive_velocity = c1 * r1 * (self.best_position - self.position)
        social_velocity = c2 * r2 * (global_best_position - self.position)
        self.velocity = w * self.velocity + cognitive_velocity + social_velocity

    def update_position(self):
        self.position += self.velocity
        self.position = np.clip(self.position, -5.12, 5.12)  # Keep within bounds


In [5]:
def pso(opt_function, dim, num_particles=30, max_iter=100, w=0.5, c1=2.05, c2=2.05):
    particles = [Particle(dim) for _ in range(num_particles)]
    global_best_value = float('inf')
    global_best_position = None
    
    for it in range(max_iter):
        for particle in particles:
            fitness_value = opt_function(particle.position)
            
            if fitness_value < particle.best_value:
                particle.best_value = fitness_value
                particle.best_position = np.copy(particle.position)
                
            if fitness_value < global_best_value:
                global_best_value = fitness_value
                global_best_position = np.copy(particle.position)
                
        for particle in particles:
            ####### Now uses default values for w, c1, and c2 if not given
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
            
    return global_best_position, global_best_value

In [6]:
# 2D
best_position_2d, best_value_2d = pso(rastrigin, 2)
print("Best Position 2D:", best_position_2d)
print("Best Value 2D:", best_value_2d)

# 3D
best_position_3d, best_value_3d = pso(rastrigin, 3)
print("Best Position 3D:", best_position_3d)
print("Best Value 3D:", best_value_3d)

#  4D
best_position_4d, best_value_4d = pso(rastrigin, 4)
print("Best Position 4D:", best_position_4d)
print("Best Value 4D:", best_value_4d)

Best Position 2D: [-4.64677126e-09 -8.39518866e-09]
Best Value 2D: 1.7763568394002505e-14
Best Position 3D: [-7.36117581e-05 -7.68402040e-05 -9.94777108e-01]
Best Value 3D: 0.9949678377610454
Best Position 4D: [ 1.35061892e-03  6.51002454e-05  1.05889196e-02 -3.12394971e-02]
Best Value 4D: 0.21559371243623104


## Experiments 1
With 100 and 200 iterations, 30 and 50 particles, in each dimension. 

In [7]:
import pandas as pd

In [8]:
def run_experiments(dimensions, experiments, pso_func):
    results = []
    for exp in experiments:
        _, best_value = pso_func(rastrigin, dim=dimensions, num_particles=exp['particles'], max_iter=exp['iterations'])
        results.append({
            'Dimensions': dimensions,
            'Particles': exp['particles'],
            'Iterations': exp['iterations'],
            'Best Value': best_value
        })
    return pd.DataFrame(results)

In [14]:
####### Config
experiments_config = [
    {'particles': 30, 'iterations': 100},
    {'particles': 50, 'iterations': 100},
    {'particles': 30, 'iterations': 250},
    {'particles': 50, 'iterations': 250},
]

####### Run
results_2d = run_experiments(2, experiments_config, pso)
results_3d = run_experiments(3, experiments_config, pso)
results_4d = run_experiments(4, experiments_config, pso)

####### Result
print("Results for 2D:")
print(results_2d.to_string(index=False))
print("\nResults for 3D:")
print(results_3d.to_string(index=False))
print("\nResults for 4D:")
print(results_4d.to_string(index=False))

Results for 2D:
 Dimensions  Particles  Iterations   Best Value
          2         30         100 5.649525e-11
          2         50         100 5.329071e-13
          2         30         250 0.000000e+00
          2         50         250 0.000000e+00

Results for 3D:
 Dimensions  Particles  Iterations   Best Value
          3         30         100 3.195693e-01
          3         50         100 4.282433e-07
          3         30         250 0.000000e+00
          3         50         250 0.000000e+00

Results for 4D:
 Dimensions  Particles  Iterations   Best Value
          4         30         100 4.058103e-04
          4         50         100 9.952837e-01
          4         30         250 7.958079e-13
          4         50         250 0.000000e+00


## Experiments 2
With 100 and 200 iterations, 30 and 50 particles, in each dimension. 

The inertia weight (w), cognitive acceleration factor (c1), and social acceleration factor (c2) become dynamic.

In [15]:
def pso_dynamic_params(opt_function, dim, num_particles=30, max_iter=100, w_start=0.9, w_end=0.4, c1_start=1.5, c1_end=2.5, c2_start=1.5, c2_end=2.5):
    particles = [Particle(dim) for _ in range(num_particles)]
    global_best_value = float('inf')
    global_best_position = None
    
    for it in range(max_iter):
        ####### Dynamically adapt parameters
        w = w_start - (w_start - w_end) * (it / max_iter)
        c1 = c1_start + (c1_end - c1_start) * (it / max_iter)
        c2 = c2_start + (c2_end - c2_start) * (it / max_iter)
        
        for particle in particles:
            fitness_value = opt_function(particle.position)
            
            if fitness_value < particle.best_value:
                particle.best_value = fitness_value
                particle.best_position = np.copy(particle.position)
                
            if fitness_value < global_best_value:
                global_best_value = fitness_value
                global_best_position = np.copy(particle.position)
                
        for particle in particles:
            particle.update_velocity(global_best_position, w, c1, c2)
            particle.update_position()
            
    return global_best_position, global_best_value

In [16]:
####### Config
experiments_config_dynamic = [
    {'particles': 30, 'iterations': 100},
    {'particles': 50, 'iterations': 100},
    {'particles': 30, 'iterations': 250},
    {'particles': 50, 'iterations': 250},
]

####### Run
results_2d_dynamic = run_experiments(2, experiments_config_dynamic, pso_dynamic_params)
results_3d_dynamic = run_experiments(3, experiments_config_dynamic, pso_dynamic_params)
results_4d_dynamic = run_experiments(4, experiments_config_dynamic, pso_dynamic_params)

####### Result
print("Dynamic Params Results for 2D:")
print(results_2d_dynamic.to_string(index=False))
print("\nDynamic Params Results for 3D:")
print(results_3d_dynamic.to_string(index=False))
print("\nDynamic Params Results for 4D:")
print(results_4d_dynamic.to_string(index=False))


Dynamic Params Results for 2D:
 Dimensions  Particles  Iterations   Best Value
          2         30         100 2.884665e-06
          2         50         100 1.041603e-07
          2         30         250 0.000000e+00
          2         50         250 0.000000e+00

Dynamic Params Results for 3D:
 Dimensions  Particles  Iterations   Best Value
          3         30         100 1.560659e+00
          3         50         100 1.070602e-04
          3         30         250 6.806999e-12
          3         50         250 3.435474e-11

Dynamic Params Results for 4D:
 Dimensions  Particles  Iterations  Best Value
          4         30         100    2.011289
          4         50         100    1.333124
          4         30         250    0.995080
          4         50         250    0.041595
